In [2]:
# Pandas
import pandas as pd

In [9]:
fileName = '[FAASA2104]_Work Master@20220211131621_All_qlqh.xlsx'
sheetName ='Work Master'
headerRow = 2
naValues = '-'

In [10]:
pd.read_excel(fileName, sheet_name = sheetName, header= headerRow, na_values= naValues)

,Work Master\nCode,Discipline,Category(Large),Unnamed: 3,Category(Middle),Unnamed: 5,Category(Small),Unnamed: 7,Attributes Specifications,Unnamed: 9,...,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,UoM,Unnamed: 34,Work Group\nCode,Work Master\nCode.1
0,NaN,NaN,Code,Description,Code,Description,Code,Description,Code,L04,...,B04,B05,B06,B07,B08,B09,UoM1,UoM2,NaN,NaN
1,A00ZZ000-00001,AR,A00,AR-Dummy,ZZ,NaN,000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOT,LOT,A00ZZ000,A00ZZ000-00001
2,A01ZZ001-00001,AR,A01,Earth Work,ZZ,NaN,001,Excavation,01,"Soil, Manual",...,NaN,NaN,NaN,NaN,NaN,NaN,M3,M3,A01ZZ001,A01ZZ001-00001
3,A01ZZ001-00002,AR,A01,Earth Work,ZZ,NaN,001,Excavation,01,"Soil, Manual",...,NaN,NaN,NaN,NaN,NaN,NaN,M3,M3,A01ZZ001,A01ZZ001-00002
4,A01ZZ001-00003,AR,A01,Earth Work,ZZ,NaN,001,Excavation,01,"Soil, Manual",...,NaN,NaN,NaN,NaN,NaN,NaN,M3,M3,A01ZZ001,A01ZZ001-00003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1620,S04AE015-00001,SS,S04,Miscellaneous Steel Erection Work,AE,Structural Steel,015,Steel Painting,NaN,NaN,...,NaN,Material: ( ),NaN,NaN,NaN,NaN,M2,M2,S04AE015,S04AE015-00001
1621,S05AF032-00001,SS,S05,Demolition Work,AF,Revamping Work (Shelter/Building),032,Demolition Work of Steel for Building/Shelter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,TON,TON,S05AF032,S05AF032-00001
1622,S05AG033-00001,SS,S05,Demolition Work,AG,Demolition Work,033,Structural Steel for Piperack/ Equipment struc...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,TON,TON,S05AG033,S05AG033-00001
1623,S05AG034-00001,SS,S05,Demolition Work,AG,Demolition Work,034,Miscellaneous Steel for Piperack/ Equipment st...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,TON,TON,S05AG034,S05AG034-00001


In [13]:
# %%writefile Component.py

from dataclasses import dataclass
from typing import List

@dataclass
class Position:
    x: float = 0
    y: float = 0
    z: float = 0

    def __add__(self, pos):
        return Position(self.x + pos.x, self.y + pos.y, self.z + pos.z)

    def __iadd__(self, pos):
        self.x += pos.x
        self.y += pos.y
        self.z += pos.z
        return self


class Component:
    def __init__(self, name: str, pos: Position):
        self._name: str = name
        self._position: Position = pos

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value: Position):
        self._name = value

    @property
    def position(self):
        return self._position

    @position.setter
    def position(self, value: Position):
        self._position = value

    def get_composite(self):
        raise NotImplementedError

    def move(self, delta: Position):
        raise NotImplementedError

    def scale(self, factor: float):
        raise NotImplementedError

class Composite(Component):
    def __init__(self, name: str, pos: Position):
        super().__init__(name, pos)
        self._leafs: List[Component] = []

    def get_composite(self):
        return self

    def move(self, delta: Position):
        for leaf in self._leafs:
            leaf.move(delta)

    def scale(self, factor: float):
        for leaf in self._leafs:
            leaf.scale(factor)

    def add_component(self, comp: Component):
        self._leafs.append(comp)

    def remove_component(self, comp: Component):
        self._leafs.remove(comp)

    def get_childs(self) -> List[Component]:
        return self._leafs

class Leaf(Component):
    def __init__(self, name: str, pos: Position):
        super().__init__(name, pos)

    def get_composite(self):
        return None

    def move(self, delta: Position):
        self.position += delta
        print(f"{self.name} move {self.position}")

    def scale(self, factor: float):
        print(f"{self.name} scale : {factor}")

Writing Component.py


In [15]:
from Component import *

def main():
    collection1 = Composite("Collection1", Position(1, 1, 1))
    cylinder = Leaf("Cylinder", Position(0, 0, 0))
    collection2 = Composite("Collection2", Position(0, 0, 0))
    box = Leaf("Box", Position(0, 0, 0))
    camera = Leaf("Camera", Position(10, 10, 3))
    light = Leaf("Light", Position(15, 10, 13))

    collection2.add_component(camera)
    collection2.add_component(light)
    collection2.add_component(box)
    for item in collection2.get_childs():
        print(item.name)

    collection1.add_component(cylinder)
    collection1.add_component(collection2)
    for item in collection1.get_childs():
        print(item.name)

    box.move(Position(2, 2, 2))
    collection1.scale(1.3)

In [16]:
main()

Camera
Light
Box
Cylinder
Collection2
Box move Position(x=2, y=2, z=2)
Cylinder scale : 1.3
Camera scale : 1.3
Light scale : 1.3
Box scale : 1.3
